# Modelo para Beta Bank

## Índice <a id='back'></a>
* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
    * [1. 1. Valores Ausentes](#data_review_missing)
    * [1. 2. Codificación de datos categóricos](#data_review_categoric)
    * [1. 3. Conclusiones](#data_review_conclusions)
* [Etapa 2. Segmentación de los datos](#data_segmentation)
    * [2. 1. Estandarización de datos numéricos](#data_segmentation_standard)
* [Etapa 3. Elección del mejor modelo](#data_model)
    * [3.1. El mejor modelo con desequilibrio de clases](#data_model_imbalance)
    * [3.2. Ajuste de peso de clase](#data_model_weight)
    * [3.3. Sobremuestreo](#data_model_upsampling)
    * [3.4. Conclusiones](#data_model_conclusions)
* [Etapa 4. Prueba final](#data_test)
* [Etapa 5. Conclusiones](#data_conclusion)

## Introducción <a id='intro'></a>

Beta Bank está perdiendo clientes cada mes, poco a poco. El banco descubrió que es más barato salvar a los clientes existentes que atraer nuevos. Se cuenta con los datos sobre el comportamiento de los clientes y su terminación de contrato con el banco.

**Objetivo**

El objetivo de este proyecto es crear un modelo con el máximo valor *F1* posible. Este valor debe ser mayor o igual que 0.59.

**Etapas**
Los datos se almacenan en el archivo `/datasets/Churn.csv`. Como se desconoce la calidad de los datos, el proyecto consistirá en cinco etapas:

1. Descripción y preprocesamiento de los datos.
2. Segmentación de los datos.
3. Elección del mejor modelo.
4. Prueba final.
5. Conclusión general.

[Volver a Contenidos](#back)

## Etapa 1. Descripción de los datos <a id='data_review'></a>

Se importan las librerías necesarias.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score, mean_squared_error, r2_score, mean_absolute_error
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

Se lee el archivo y se guardará en la variable `df`.

In [2]:
try:
    df = pd.read_csv('Churn.csv')
except:
    df = pd.read_csv('/datasets/Churn.csv')

Se imprimirá la información general de `df` y las primeras 10 filas.

In [3]:
df.info()
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age   
0          1    1563

El dataset cuenta con 10,000 filas, cada una de las cuales contiene información del comportamiento sobre un cliente. Las 13 columnas de características son:

1. `'RowNumber'` - índice de cadena de datos.
2. `'CustomerId'` - Identificador único de cliente.
3. `'Surname'` - apellido del cliente.
4. `'CreditScore'` - valor de crédito del cliente.
5. `'Geography'` - País de residencia.
6. `'Gender'` - sexo.
7. `'Age'` - edad.
8. `'Tenure'` - período durante el cual ha madurado el depósito a plazo fijo del cliente (en años).
9. `'Balance'` - saldo de la cuenta.
10. `'NumOfProducts'` - número de productos bancarios utilizados por el cliente.
11. `'HasCrCard'` - el cliente tiene una tarjeta de crédito (1 - si, 0 - no).
12. `'IsActiveMember'` - el cliente es mimbro (1 - si, 0 - no).
13. `'EstimatedSalary'` - salario estimado.

La columna objetivo `'Exited'` la cual contiene información sobre si el cliente se ha ido (1 - si, 0 - no).

Observamos que los nombres de las columnas no están de la manera usual (minúsculas y usando `_` para separar palabras), se procederá a cambiarlos. La columna `'Tenure'` tiene valores ausentes, se investigarán más adelante para tratarlos. Gracias al identificador único de cliente es sencillo comprobar si existen duplicados obvios.

In [4]:
#Nombres de las columnas del dataframe
print(df.columns)

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')


In [5]:
df.columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited']

print(df.columns)

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')


Ya que los nombres de las columnas está actualizados, se comprobará si existen duplicados obvios.

In [6]:
print(df.duplicated().sum())

0


No existen duplicados obvios. En la siguiente sección se tratarán los valores ausentes.

[Volver a Contenidos](#back)

### 1. 1. Valores ausentes <a id='data_review_missing'></a>

Se imprimirán las primeras 10 filas con valores ausentes para investigar más.

In [7]:
print(df[df['tenure'].isna()].head(10))

     row_number  customer_id    surname  credit_score geography  gender  age   
30           31     15589475    Azikiwe           591     Spain  Female   39  \
48           49     15766205        Yin           550   Germany    Male   38   
51           52     15768193  Trevisani           585   Germany    Male   36   
53           54     15702298   Parkhill           655   Germany    Male   41   
60           61     15651280     Hunter           742   Germany    Male   35   
82           83     15641732      Mills           543    France  Female   36   
85           86     15805254    Ndukaku           652     Spain  Female   75   
94           95     15676966      Capon           730     Spain    Male   42   
99          100     15633059    Fanucci           413    France    Male   34   
111         112     15665790   Rowntree           538   Germany    Male   39   

     tenure    balance  num_of_products  has_cr_card  is_active_member   
30      NaN       0.00                3      

No parece existir ninguna relación con la información de otra columna. Se imprimiran las estadísticas de esta columna.

In [8]:
print(df['tenure'].describe())

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: tenure, dtype: float64


In [9]:
#porcentaje de valores ausentes
print(len(df[df['tenure'].isna()]) / len(df) * 100)

9.09


Como es el 9% de los datos, se decide reemplazar los valores ausentes por la mediana.

In [10]:
median = df['tenure'].median()
df = df.fillna(median)
print(df.isna().sum())

row_number          0
customer_id         0
surname             0
credit_score        0
geography           0
gender              0
age                 0
tenure              0
balance             0
num_of_products     0
has_cr_card         0
is_active_member    0
estimated_salary    0
exited              0
dtype: int64


Ya no existen valores ausentes. En la siguiente sección se preparán los datos para el modelo.

[Volver a Contenidos](#back)

### 1. 2. Codificación de datos categóricos<a id='data_review_categoric'></a>

Existen columnas que no aportarán información relevante al modelo, las cuales son:

* `'row_number'` - esta columna describe casi la misma información que el índice del dataframe, la diferencia es que comienza en 1. No es una característica para el modelo.
* `'customer_id'` - esta información identifica al cliente de manera única lo cual no es relevante para el modelo.
* `'surname'` - el apellido del cliente es otra manera de identificar lo cual tampoco es relevante para el modelo.

Se procede a eliminar estas columnas.

In [11]:
df = df.drop(['row_number', 'customer_id', 'surname'], axis=1)
print(df.head())

   credit_score geography  gender  age  tenure    balance  num_of_products   
0           619    France  Female   42     2.0       0.00                1  \
1           608     Spain  Female   41     1.0   83807.86                1   
2           502    France  Female   42     8.0  159660.80                3   
3           699    France  Female   39     1.0       0.00                2   
4           850     Spain  Female   43     2.0  125510.82                1   

   has_cr_card  is_active_member  estimated_salary  exited  
0            1                 1         101348.88       1  
1            0                 1         112542.58       0  
2            1                 0         113931.57       1  
3            0                 0          93826.63       0  
4            1                 1          79084.10       0  


Se codificarán las columnas categóricas, sólo son dos `'geography'` y `'gender'`. Para ello se imprimirá la cantidad de valores en ambas.

In [12]:
print(df['geography'].value_counts())
print(df['gender'].value_counts())

geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64
gender
Male      5457
Female    4543
Name: count, dtype: int64


En la columna `'geography'` se tienen tres opciones, mientras que en la columna `'gender'` se tienen dos. Al codificarlas aparecerán tres columnas para la primera y dos para la segunda. Sin embargo, se puede reducir eliminando la primera de cada una. Esto es, se tendrán las columnas `'geography_France'`, `'geography_Germnay'` y `'geography_Spain'`, en donde será 1 en el país de residencia y 0 en otro caso. Se observa que si el cliente reside en Francia entonces en las otras dos columnas aparecerá 0, por ello quitar la primera columna mantiene la información.

In [13]:
df = pd.get_dummies(df, drop_first=True, dtype=int)
print(df.head())

   credit_score  age  tenure    balance  num_of_products  has_cr_card   
0           619   42     2.0       0.00                1            1  \
1           608   41     1.0   83807.86                1            0   
2           502   42     8.0  159660.80                3            1   
3           699   39     1.0       0.00                2            0   
4           850   43     2.0  125510.82                1            1   

   is_active_member  estimated_salary  exited  geography_Germany   
0                 1         101348.88       1                  0  \
1                 1         112542.58       0                  0   
2                 0         113931.57       1                  0   
3                 0          93826.63       0                  0   
4                 1          79084.10       0                  0   

   geography_Spain  gender_Male  
0                0            0  
1                1            0  
2                0            0  
3               

### 1. 3. Conclusiones <a id='data_review_conclusions'></a>

En esta etapa se describieron las columnas que contiene el dataframe. No se encontraron duplicados obvios y la única columna con valores ausentes fue `'tenure'`. Se reemplazaron estos valores con la mediana.

También se codificaron los datos de tipo categórico, estos fueron los correspondientes a las columnas con la información del país de residencia y el género de cada cliente.

[Volver a Contenidos](#back)

## Etapa 2. Segmentación de los datos <a id='data_segmentation'></a>

En esta etapa se segmentará el dataset `df` en tres conjuntos, siguiendo una proporción 3:1:1. Es decir, se tendrán lo siguiente:

1. un conjunto de entrenamiento que representa el 60% de los datos,
2. un conjunto de validación que representa el 20% y
3. un conjunto de prueba que representa el 20% restante.

In [14]:
# Primero se segmenta df para obtener el conjunto de entrenamiento (60%)
df_train, df2 = train_test_split(df, test_size=0.40, random_state=12345)
# Ahora se segmenta el resto (df2) a la mitad para obtener los conjuntos de validación y de prueba, cada uno con el 20% de df
df_valid, df_test = train_test_split(df2, test_size=0.50, random_state=12345)

El conjunto de entrenamiento es `df_train`, el de validación es `df_valid` y el de prueba es `df_test`.

Se filtrarán los datasets para establecer las características y el objetivo del modelo.

In [15]:
features_train = df_train.drop(['exited'], axis=1)
target_train = df_train['exited']
features_valid = df_valid.drop(['exited'], axis=1)
target_valid = df_valid['exited']
features_test = df_test.drop(['exited'], axis=1)
target_test = df_test['exited']

### 2. 1. Estandarización de datos numéricos <a id='data_segmentation_standard'></a>

A continuación se estandarizarán los datos numéricos en el conjunto de entrenamiento, posteriormente se utilizarán estos parámetros para estandarizar los datos numéricos de los conjuntos de validación y prueba.

In [16]:
numeric = ['credit_score', 'age', 'tenure', 'balance',
           'num_of_products', 'estimated_salary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

En la siguiente etapa se buscará el mejor modelo.

[Volver a Contenidos](#back)

## Etapa 3. Elección del mejor modelo <a id='data_model'></a>

Primero se investigará el equilibrio de clases. Para ello se imprimirán los valores de `target_train`.

In [17]:
print(target_train.value_counts())

exited
0    4804
1    1196
Name: count, dtype: int64


El porcentaje de clientes que no se han ido es aproximadamente del 80%, es quiere decir que se tiene una proporción de 4:1. Las clases están desequilibradas.

Se buscará el mejor modelo, primero sin tomar en cuenta el desequilibrio de las clases y en caso de no superar el 0.59 en el valor F1, entonces se ajustará este desequilibrio.

### 3. 1. El mejor modelo con desequilibrio de clases <a id='data_model_imbalance'></a>

El primer modelo se creará con un árbol de decisión y se ajustará la profundidad máxima para obtener el valor F1 más alto.

In [18]:
for i in range(1, 15):
    model = DecisionTreeClassifier(random_state=54321, max_depth=i)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    f1 = round(f1_score(target_valid, predictions), 3)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = round(roc_auc_score(target_valid, probabilities_one_valid), 3)
    print(f'max_depth = {i}, F1: {f1}, AUC-ROC: {auc_roc}')

max_depth = 1, F1: 0.0, AUC-ROC: 0.693
max_depth = 2, F1: 0.522, AUC-ROC: 0.75
max_depth = 3, F1: 0.423, AUC-ROC: 0.797
max_depth = 4, F1: 0.553, AUC-ROC: 0.813
max_depth = 5, F1: 0.541, AUC-ROC: 0.823
max_depth = 6, F1: 0.569, AUC-ROC: 0.816
max_depth = 7, F1: 0.538, AUC-ROC: 0.815
max_depth = 8, F1: 0.543, AUC-ROC: 0.81
max_depth = 9, F1: 0.562, AUC-ROC: 0.781
max_depth = 10, F1: 0.537, AUC-ROC: 0.767
max_depth = 11, F1: 0.52, AUC-ROC: 0.735
max_depth = 12, F1: 0.518, AUC-ROC: 0.709
max_depth = 13, F1: 0.505, AUC-ROC: 0.691
max_depth = 14, F1: 0.501, AUC-ROC: 0.688


El valor F1 mayor es con profundidad máxima 6, el cual es igual a 0.569 con un valor AUC-ROC igual a 0.816. Este valor F1 no supera 0.59.

Se probará con un modelo creado con un bosque aleatorio. Se ajustará tanto la profundidad como el número de árboles.

In [19]:
best_F1 = 0
best_est = 0
best_depth = 0
for i in range(1, 11): #rango de profundidad
    for est in range(10, 101, 10): # rango para los ábroles, comenzando en 10, terminando en 100 y dando saltos de 10 en 10
        model = RandomForestClassifier(random_state=54321, max_depth=i, n_estimators=est) # n_estimators ajusta el número de árboles
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > best_F1:
            best_F1 = f1
            best_est = est
            best_depth = i            

print("F1 del mejor modelo en el conjunto de validación (n_estimators = {}, max_depth = {}): {}".format(best_est, best_depth, best_F1))

F1 del mejor modelo en el conjunto de validación (n_estimators = 10, max_depth = 9): 0.5984962406015036


El mejor modelo es con 10 árboles y profundidad máxima igual a 9. Se calculará el valor AUC-ROC.

In [20]:
rf_model = RandomForestClassifier(random_state=54321, max_depth=9, n_estimators=10)
rf_model.fit(features_train, target_train)
rf_predictions = rf_model.predict(features_valid)
f1 = f1_score(target_valid, rf_predictions)
rf_probabilities_valid = rf_model.predict_proba(features_valid)
rf_probabilities_one_valid = rf_probabilities_valid[:, 1]
rf_auc_roc = roc_auc_score(target_valid, rf_probabilities_one_valid)

print(f'F1 = {f1}, AUC-ROC = {rf_auc_roc}')

F1 = 0.5984962406015036, AUC-ROC = 0.8389431946721188


En este caso, el valor F1 cumple con lo esperado (es mayor o igual que 0.59) y el valor AUC-ROC es cercano a 1.

A continuación, se examinará un modelo creado con regresión logística.

In [21]:
rl_model = LogisticRegression(random_state=54321, solver='liblinear')
rl_model.fit(features_train, target_train)
rl_predictions = rl_model.predict(features_valid)
rl_f1 = f1_score(target_valid, rl_predictions)
rl_probabilities_valid = rl_model.predict_proba(features_valid)
rl_probabilities_one_valid = rl_probabilities_valid[:, 1]
rl_auc_roc = roc_auc_score(target_valid, rl_probabilities_one_valid)

print(f'F1 = {rl_f1}, AUC-ROC = {rl_auc_roc}')

F1 = 0.33108108108108103, AUC-ROC = 0.7587512627102753


Con este modelo el valor F1 decayó a 0.33, también el valor AUC-ROC decayó a 0.75. Se decide que el mejor modelo es el creado con el algoritmo de bosque aleatorio. Se tomará este modelo para ajustar el desequilibrio de clases en la siguiente sección.

[Volver a Contenidos](#back)

### 3. 2. Ajuste de peso de clase <a id='data_model_weight'></a>

Se tiene que la clase rara es el 1, entonces se equilibrarán los pesos de clase balanceándolos.

In [22]:
rf_model = RandomForestClassifier(random_state=54321, max_depth=9, n_estimators=10, class_weight='balanced')
rf_model.fit(features_train, target_train)
rf_predictions = rf_model.predict(features_valid)
f1 = f1_score(target_valid, rf_predictions)
rf_probabilities_valid = rf_model.predict_proba(features_valid)
rf_probabilities_one_valid = rf_probabilities_valid[:, 1]
rf_auc_roc = roc_auc_score(target_valid, rf_probabilities_one_valid)

print(f'F1 = {f1}, AUC-ROC = {rf_auc_roc}')

F1 = 0.6077097505668935, AUC-ROC = 0.8410194835439362


Como se observa, el valor F1 aumentó a 0.60, así como el valor AUC-ROC aumentó a 0.84. En la siguiente sección se utilizará el enfoque de sobremuestreo para equilibrar las clases.

[Volver a Contenidos](#back)

### 3. 3. Sobremuestreo <a id='data_model_upsampling'></a>

En esta sección se utilizará la técnica del sobremuestreo para verificar si se obtiene un mejor modelo que en la sección anterior. Primero se creará una función que realice todo el procedimiento.

In [23]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=54321
    )
    return features_upsampled, target_upsampled

Se utilizará esta función para averiguar el mayor valor F1 posible cuando se varía `repeat`.

In [24]:
for i in range(1, 11):
    features_upsampled, target_upsampled = upsample(
    features_train, target_train, i
    )
    ups_model = RandomForestClassifier(random_state=54321, max_depth=9, n_estimators=10)
    ups_model.fit(features_upsampled, target_upsampled)
    ups_predictions = ups_model.predict(features_valid)
    ups_f1 = f1_score(target_valid, ups_predictions)
    print(f'Repeat = {i}, F1={ups_f1}')

Repeat = 1, F1=0.5813253012048193
Repeat = 2, F1=0.6062992125984252
Repeat = 3, F1=0.6109175377468061
Repeat = 4, F1=0.601472134595163
Repeat = 5, F1=0.6011673151750972
Repeat = 6, F1=0.5696316262353999
Repeat = 7, F1=0.5542372881355931
Repeat = 8, F1=0.5516102394715111
Repeat = 9, F1=0.5373831775700935
Repeat = 10, F1=0.5210332103321033


El mayor valor F1 es con 3, aumenta a 0.61. Se calculará el valor AUC-ROC.

In [25]:
features_upsampled, target_upsampled = upsample(
    features_train, target_train, 3
)

In [26]:
ups_model = RandomForestClassifier(random_state=54321, max_depth=9, n_estimators=10)
ups_model.fit(features_upsampled, target_upsampled)
ups_predictions = ups_model.predict(features_valid)
ups_f1 = f1_score(target_valid, ups_predictions)
ups_probabilities_valid = ups_model.predict_proba(features_valid)
ups_probabilities_one_valid = ups_probabilities_valid[:, 1]
ups_auc_roc = roc_auc_score(target_valid, ups_probabilities_one_valid)

print(f'F1 = {ups_f1}, AUC-ROC = {ups_auc_roc}')

F1 = 0.6109175377468061, AUC-ROC = 0.8375655550783636


En comparación con la técnica de ajuste de peso, el valor AUC-ROC bajó, sin sembargo la diferencia es mínima. Se decide usar este modelo como el mejor.

[Volver a Contenidos](#back)

### 3. 4. Conclusiones <a id='data_model_conclusions'></a>

En esta estapa se encontró que el mejor algortimo para entrenar el modelo es el bosque aleatorio con 10 árboles y profundidad máxima igual a 9. Esto se hizo sin tomar en cuenta el desequilibrio de clases.

Posteriormente se contempló este desequilibrio y para mejorar la calidad del modelo se compararon dos técnicas de equilibrio. La primera fue el ajuste de peso de clase y la segunda el sobremuestro. Esta última técnica arrojó un valor F1 mayor, por lo que se decide usar este modelo para el conjunto de dato de prueba.

[Volver a Contenidos](#back)

## Etapa 4. Prueba final <a id='data_test'></a>

En esta etapa se hará la prueba final del modelo elegido con el conjunto de datos de prueba.

In [27]:
test_predictions = ups_model.predict(features_test)
test_f1 = f1_score(target_test, test_predictions)
test_probabilities_valid = ups_model.predict_proba(features_test)
test_probabilities_one_valid = test_probabilities_valid[:, 1]
test_auc_roc = roc_auc_score(target_test, test_probabilities_one_valid)

print(f'F1 = {test_f1}, AUC-ROC = {test_auc_roc}')

F1 = 0.5934314835787089, AUC-ROC = 0.8411316036823665


El valor F1 bajó a 0.59, sin embargo sigue siendo mayor o igual a lo esperado. El modelo es aceptable.

## Etapa 5. Conclusión general <a id='data_conclusion'></a>

El conjunto de datos se dividió en tres partes: conjunto de entrenamiento (60%), conjunto de validación (20%) y conjunto de prueba (20%). Con los datos del conjunto de entrenamiento se crearon y entrenaron los modelos, usando los tres tipos de algoritmos de clasificación: árbol de decisión, bosque aleatorio y regresión lógistica, sin tomar en cuenta el desequilibrio de clases.

Para elegir el mejor modelo se ajustaron hiperparámetros y se comprobó el valor F1 de cada uno de los modelos con los datos del conjunto de validación. En el árbol de decisión se concluyó que el mayor valor F1 se obtuvó con profundidad máxima igual a 6, mientras que para el bosque aleatorio, se alcanzó el mayor valor F1 con 10 árboles y profundidad máxima igual a 9.

Comparando el valor F1 de los tres algoritmos, se concluyó que el mejor modelo es el bosque aleatorio (10 árboles y produndidad máxima 9).

Posterior a esto, se equilibraron las clases, comparando dos técnicas resultando mejor el sobremuestreo. Finalmente, se comprobó la calidad del modelo con los datos del conjunto de prueba, obteniendo un valor F1 igual a 0.59, diferencia mínima con respecto al valor F1 en el conjunto de validación. Esta cantidad concluye que el modelo cumple el objetivo.

[Volver a Contenidos](#back)